**行业板块研报爬取**

In [6]:
import requests
import time
import pandas as pd
import numpy as np
import threading
import os
import random
import csv
import json
import math
import re
import statistics
import urllib.request
from datetime import datetime,timedelta
from concurrent.futures import ThreadPoolExecutor,as_completed
from bs4 import BeautifulSoup

测试

In [7]:
#研报列表
params = {
    'industryCode': '545',
    'pageSize': 50,
    'industry': '*',
    'rating': '*',
    'ratingChange': '*',
    'beginTime': '2021-07-06',
    'endTime': '2023-07-06',
    'pageNo': 1,
    'fields': '',
    'qType': 1,
    'orgCode': '',
    'rcode': ''
}
response = requests.get('https://reportapi.eastmoney.com/report/list?',params=params).json()
response['data'][0]

{'title': '科学仪器行业双周谈第二期：多维度剖析色谱仪器产业链，看国内优质公司发展空间',
 'stockName': '',
 'stockCode': '',
 'orgCode': '80000162',
 'orgName': '开源证券股份有限公司',
 'orgSName': '开源证券',
 'publishDate': '2023-07-05 00:00:00.000',
 'infoCode': 'AP202307051592048199',
 'column': '002002003002',
 'predictNextTwoYearEps': '',
 'predictNextTwoYearPe': '',
 'predictNextYearEps': '',
 'predictNextYearPe': '',
 'predictThisYearEps': '',
 'predictThisYearPe': '',
 'predictLastYearEps': '',
 'predictLastYearPe': '',
 'actualLastTwoYearEps': '',
 'actualLastYearEps': '',
 'industryCode': '545',
 'industryName': '通用设备',
 'emIndustryCode': '101009',
 'indvInduCode': '',
 'indvInduName': '',
 'emRatingCode': '',
 'emRatingValue': '',
 'emRatingName': '',
 'lastEmRatingCode': '',
 'lastEmRatingValue': '',
 'lastEmRatingName': '',
 'ratingChange': '',
 'reportType': 3,
 'author': ['11000170965.诸海滨', '11000330948.赵昊'],
 'indvIsNew': '',
 'researcher': '诸海滨,赵昊',
 'newListingDate': '',
 'newPurchaseDate': '',
 'newIssuePrice': '',
 

In [8]:
#研报内容
infoCode = 'AP202208081576981076'
html = requests.get(f'https://data.eastmoney.com/report/zw_industry.jshtml?infocode={infoCode}').text
soup = BeautifulSoup(html,'html.parser')
soup

<!DOCTYPE html>

<!-- 2023-07-25 17:41:56 at server[193] -->
<html lang="zh_CN">
<head>
<meta charset="utf-8"/>
<meta content="玻璃制造行业研究周报：浮法玻璃部分推涨迎去库，盈利有所回升,机构研究报告,研报中心,研报,个股评级,买入评级,个股研报,盈利预测,行业研究,行业研报,策略研究" name="keywords">
<meta content="东方财富网研报中心:第一时间提供各大券商研究所报告，最大程度减少个人投资者与机构之间信息上的差异，使个人投资者更早的了解到上市公司基本面变化。" name="description"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="format=html5; url=https://emdata.eastmoney.com/ybzx/index.html" name="mobile-agent"/>
<meta content="always" name="referrer"/>
<title>玻璃制造行业研究周报：浮法玻璃部分推涨迎去库，盈利有所回升-研究报告正文 _ 数据中心 _ 东方财富网</title>
<link href="/newstatic/css/main.css" rel="stylesheet"/>
<link href="/newstatic/css/report.css" rel="stylesheet"/>
<style>
    .zw_table .vbottom{
        vertical-align: bottom;
    }
</style>
<base target="_blank"/>
<!--[if lte IE 11 ]>
    <script src="//emres.dfcfw.com/ie6/ie6.js" charset="utf-8"></script>
    <![endif]-->
<s

In [38]:
def crawl_info(code,page,file_path):
    if page%10 == 0:
        print(f"正在爬取:{code}....已爬取{page}页")
    
    #数据获取
    params = {
    'industryCode': code,
    'pageSize': 50,
    'industry': '*',
    'rating': '*',
    'ratingChange': '*',
    'beginTime': '2022-07-09',
    'endTime': '2023-07-09',
    'pageNo': page,
    'fields': '',
    'qType': 1,
    'orgCode': '',
    'rcode': ''
    }
    response = requests.get('https://reportapi.eastmoney.com/report/list?',params=params).json()
    data = response['data']
    pageNum = response['TotalPage']
    
    if pageNum != 0:
        #数据解析
        df = pd.DataFrame(data)
        #筛选列
        columns = ['publishDate','industryName','industryCode','title','orgSName','orgCode', 'lastEmRatingName', 'emRatingName','researcher','infoCode']
        df = df[columns]
        df['publishDate'] = pd.to_datetime(df['publishDate'])
        df['infoCode'] = df['infoCode'].apply(lambda x:f'https://data.eastmoney.com/report/zw_industry.jshtml?infocode={x}')
        df.to_csv(file_path,mode='a',header=False,index=False,encoding='utf-8-sig')
        #递归爬取下一页
        if(pageNum>1 and page<pageNum):
            crawl_info(code,page+1,file_path)

def crawl_content(index,info,file_path):
    global start
    if index%100 == 0:
        point = time.time()
        print(f'正在爬取第{index}条研报，用时{point - start:.2f}秒')
    url = info['研报网址']
    html = requests.get(url).text
    #解析数据
    soup = BeautifulSoup(html,'html.parser')
    pdf_link = soup.find('a',class_ = 'pdf-link').get('href')
    #保存记录
    record = info.drop('研报网址')
    record['pdf_link'] = pdf_link
    df = pd.DataFrame(record).T
    df.to_csv(file_path,mode='a',header=False,index=False,encoding='utf-8-sig')

def multi_thread(infos,file_path):
    with ThreadPoolExecutor(max_workers=20) as pool:
        futures = [pool.submit(crawl_content,index+1,info,file_path) for index,info in infos.iterrows()]

爬取研报基本信息

In [40]:
if __name__ == '__main__':
    folder_path = os.path.join(os.path.dirname(os.getcwd()),'行业数据\\研报')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    #研报基本信息爬取
    industry_data = pd.read_csv("数据/行业板块数据.csv",encoding='gbk')
    industry_code = industry_data['板块代码'].apply(lambda x:re.search(r'[1-9]\d*',x).group())
    file_name = '行业研报基本信息.csv'
    file_path = os.path.join(folder_path,file_name)
    info_columns=['发布日期','行业名称','行业代码','标题','机构名称','机构代码','东财上次评级', '东财评级','作者','研报网址']
    with open(file_path, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(info_columns) 
    start = time.time()
    print('开始爬取行业研报基本信息')
    for code in industry_code:
        crawl_info(code,1,file_path)
    end = time.time()
    print(f'行业研报基本信息爬取成功，用时{end-start:.2f}秒')

开始爬取行业研报基本信息
正在爬取:459....已爬取10页
正在爬取:459....已爬取20页
正在爬取:448....已爬取10页
正在爬取:735....已爬取10页
正在爬取:1029....已爬取10页
正在爬取:1029....已爬取20页
正在爬取:451....已爬取10页
正在爬取:451....已爬取20页
正在爬取:451....已爬取30页
正在爬取:447....已爬取10页
正在爬取:545....已爬取10页
正在爬取:486....已爬取10页
正在爬取:486....已爬取20页
正在爬取:456....已爬取10页
正在爬取:481....已爬取10页
正在爬取:538....已爬取10页
正在爬取:482....已爬取10页
正在爬取:480....已爬取10页
正在爬取:1034....已爬取10页
正在爬取:1034....已爬取20页
正在爬取:433....已爬取10页
正在爬取:728....已爬取10页
正在爬取:1019....已爬取10页
正在爬取:424....已爬取10页
正在爬取:427....已爬取10页
正在爬取:1031....已爬取10页
正在爬取:437....已爬取10页
正在爬取:438....已爬取10页
正在爬取:438....已爬取20页
正在爬取:438....已爬取30页
正在爬取:738....已爬取10页
正在爬取:1033....已爬取10页
正在爬取:478....已爬取10页
正在爬取:478....已爬取20页
正在爬取:475....已爬取10页
正在爬取:465....已爬取10页
正在爬取:485....已爬取10页
正在爬取:485....已爬取20页
正在爬取:1042....已爬取10页
行业研报基本信息爬取成功，用时47.34秒


爬取研报摘要及pdf链接

In [33]:
if __name__ == '__main__':
    folder_path = os.path.join(os.path.dirname(os.getcwd()),'研报')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    #研报内容爬取
    file_name = '行业研报摘要.csv'
    file_path = os.path.join(folder_path,file_name)
    content_columns=['发布日期','行业名称','行业代码','标题','PDF原文链接']
    with open(file_path, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(content_columns)
    
    industry_info = pd.read_csv("../行业数据\\研报/行业研报基本信息.csv",encoding='utf-8-sig')
    industry_info = industry_info[['发布日期','行业名称','行业代码','标题','研报网址']]
    start = time.time()
    print('开始爬取行业研报内容')
    multi_thread(industry_info,file_path)
    end = time.time()
    print(f'行业研报内容爬取成功，用时{end-start:.2f}秒')

开始爬取行业研报内容
正在爬取第100条研报，用时5.45秒
正在爬取第200条研报，用时11.47秒
正在爬取第300条研报，用时18.74秒
正在爬取第400条研报，用时24.60秒
正在爬取第500条研报，用时30.83秒
正在爬取第600条研报，用时37.23秒
正在爬取第700条研报，用时43.05秒
正在爬取第800条研报，用时48.79秒
正在爬取第900条研报，用时54.18秒
正在爬取第1000条研报，用时59.83秒
正在爬取第1100条研报，用时65.35秒
正在爬取第1200条研报，用时70.89秒
正在爬取第1300条研报，用时76.34秒
正在爬取第1400条研报，用时81.77秒
正在爬取第1500条研报，用时87.31秒
正在爬取第1600条研报，用时92.98秒
正在爬取第1700条研报，用时98.65秒
正在爬取第1800条研报，用时104.08秒
正在爬取第1900条研报，用时274.18秒
正在爬取第2000条研报，用时280.61秒
正在爬取第2100条研报，用时286.62秒
正在爬取第2200条研报，用时292.40秒
正在爬取第2300条研报，用时298.43秒
正在爬取第2400条研报，用时304.19秒
正在爬取第2500条研报，用时310.49秒
正在爬取第2600条研报，用时316.63秒
正在爬取第2700条研报，用时322.58秒
正在爬取第2800条研报，用时328.51秒
正在爬取第2900条研报，用时335.06秒
正在爬取第3000条研报，用时340.73秒
正在爬取第3100条研报，用时346.46秒
正在爬取第3200条研报，用时352.21秒
正在爬取第3300条研报，用时358.24秒
正在爬取第3400条研报，用时363.98秒
正在爬取第3500条研报，用时369.71秒
正在爬取第3600条研报，用时375.38秒
正在爬取第3700条研报，用时381.40秒
正在爬取第3800条研报，用时387.19秒
正在爬取第3900条研报，用时393.03秒
正在爬取第4000条研报，用时398.83秒
正在爬取第4100条研报，用时405.11秒
正在爬取第4200条研报，用时410.67秒
正在爬取第4300条研报，用时416.68秒
正在爬取第4400条研报，用时422.25秒
正在爬取

下载pdf

In [41]:
if __name__ == '__main__':
    #pdf下载
    pdf_df = pd.read_csv('../研报/行业研报摘要.csv')
    industry_data = pd.read_csv("数据/行业板块数据.csv",encoding='gbk')
    industry_name = industry_data['板块名称']
    print('开始下载研报原文')
    start = time.time()
    for name in industry_name:
        print(f'开始下载{name}研报')
        report_list = pdf_df[pdf_df['行业名称'] == name]['PDF原文链接']
        title_list = pdf_df[pdf_df['行业名称'] == name]['标题']
        if len(report_list)!=0:
            report_group = np.array_split(report_list,4)
            title_group = np.array_split(title_list,4)
            for i in range(4):
                urls = report_group[i]
                titles = title_group[i]
                with ThreadPoolExecutor(max_workers=5) as pool:
                    for url,title in zip(urls,titles):
                        title = title.replace(':','-')
                        folder_path = os.path.join(os.path.dirname(os.getcwd()),f'行业数据/研报/研报原文/{name}')
                        if not os.path.exists(folder_path):
                            os.makedirs(folder_path)
                        file_path = os.path.join(folder_path,f'{title}.pdf')
                        future = pool.submit(urllib.request.urlretrieve,url,file_path)
                point = time.time()
                print(f"下载进度：{(i+1)*25}%,已用时{point - start:.2f}秒")
            print(f'{name}研报下载成功')
        else:
            print(f'{name}暂无研报数据')
        
    end = time.time()
    print(f'行业研报下载成功，用时{end-start:.2f}秒')

开始下载研报原文
开始下载船舶制造研报
下载进度：25%,已用时0.66秒
下载进度：50%,已用时1.33秒
下载进度：75%,已用时1.89秒
下载进度：100%,已用时2.52秒
船舶制造研报下载成功
开始下载电子元件研报
下载进度：25%,已用时20.84秒
下载进度：50%,已用时39.94秒
下载进度：75%,已用时60.24秒
下载进度：100%,已用时81.02秒
电子元件研报下载成功
开始下载通信设备研报
下载进度：25%,已用时92.84秒
下载进度：50%,已用时105.29秒
下载进度：75%,已用时128.84秒
下载进度：100%,已用时159.96秒
通信设备研报下载成功
开始下载计算机设备研报
下载进度：25%,已用时173.96秒
下载进度：50%,已用时189.26秒
下载进度：75%,已用时201.67秒
下载进度：100%,已用时215.71秒
计算机设备研报下载成功
开始下载游戏研报
下载进度：25%,已用时218.24秒
下载进度：50%,已用时220.93秒
下载进度：75%,已用时224.12秒
下载进度：100%,已用时227.33秒
游戏研报下载成功
开始下载光学光电子研报
下载进度：25%,已用时228.24秒
下载进度：50%,已用时228.57秒
下载进度：75%,已用时228.77秒
下载进度：100%,已用时228.88秒
光学光电子研报下载成功
开始下载消费电子研报
下载进度：25%,已用时230.45秒
下载进度：50%,已用时232.02秒
下载进度：75%,已用时234.34秒
下载进度：100%,已用时236.91秒
消费电子研报下载成功
开始下载半导体研报
下载进度：25%,已用时249.28秒
下载进度：50%,已用时257.50秒
下载进度：75%,已用时268.04秒
下载进度：100%,已用时277.20秒
半导体研报下载成功
开始下载通信服务研报
下载进度：25%,已用时278.92秒
下载进度：50%,已用时280.86秒
下载进度：75%,已用时283.77秒
下载进度：100%,已用时289.92秒
通信服务研报下载成功
开始下载汽车整车研报
下载进度：25%,已用时309.98秒
下载进度：50%,已用时329.40秒
下载进度：75%,已用时348.69秒
下载进度：100